# Setup: Connectivity

In [1]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


My Client: <katana_enterprise.remote.sync_wrappers.Client object at 0x7fb372949d60>
My Graph:  <_Graph my_graph, 8Wv1JJ5UT6iBJqMR1Sk4kxAxZUrEVACNWP9tdJDegBrA, 2>



#  Step 01: Enter a new Medical Description

In [2]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


Textarea(value='Odd mole on my ear', description='Enter Text: ', layout=Layout(flex='0 1 auto', height='100px'…

In [3]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Patient Visit Id: %s" % (l_pvid))
print("")
print("Medical Description encoding:")
print("")
#  display(l_return)


        id                                      transcription         LABEL
0  PV-2455  {'entityMentions': [{'mentionId': '1', 'type':...  PatientVisit
1  XX-1001                               {'x-col': 'XX-1001'}  PatientVisit
2  XX-1002                               {'x-col': 'XX-1002'}  PatientVisit
3  XX-1003                               {'x-col': 'XX-1003'}  PatientVisit
TTT
Empty DataFrame
Columns: [id, entity_id, preferred_term, LABEL]
Index: []
TTT
Empty DataFrame
Columns: [id, LABEL]
Index: []
Unique Patient Visit Id: PV-2455

Medical Description encoding:



# Step 02:  Examine the Graph

In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="sample_name"      ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary),
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, ms, ios
   
   """.format(l_pvid)

l_result = my_graph.query(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


In [ ]:

l_query  = """

   MATCH (n)
   RETURN n
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)
